1.

>



In [1]:
!pip install git+https://github.com/facebookresearch/segment-anything.git
!pip install opencv-python matplotlib
!pip install torch torchvision
!pip install scikit-learn

  Cloning https://github.com/facebookresearch/segment-anything.git to /tmp/pip-req-build-jjaj7eql
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/segment-anything.git /tmp/pip-req-build-jjaj7eql
  Resolved https://github.com/facebookresearch/segment-anything.git to commit dca509fe793f601edb92606367a655c15ac00fdf
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for segment_anything: filename=segment_anything-1.0-py3-none-any.whl size=36636 sha256=fc80314358da2938172637b43d2f54f342125655fc24c896d647a68dd6b19289
  Stored in directory: /tmp/pip-ephem-wheel-cache-j0mrcyxm/wheels/29/82/ff/04e2be9805a1cb48bec0b85b5a6da6b63f647645750a0e42d4
Successfully built segment_anything
  Using cached pillow-11.2.1-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 4.2 MB/s eta 0:00:00m eta 0:00:010:0

2.

In [2]:
import os
import numpy as np
import cv2
import torch
import matplotlib.pyplot as plt
from PIL import Image
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
from segment_anything import SamPredictor
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

In [ ]:
MODEL_TYPE = "vit_b"
MODEL_PATH = "sam_vit_b_01ec64.pth"  # Caminho para o modelo SAM


def load_sam_model(model_type=MODEL_TYPE, model_path=MODEL_PATH):
    sam = sam_model_registry[model_type](checkpoint=model_path)
    sam.to(device='cuda' if torch.cuda.is_available() else 'cpu')
    return sam

3.

In [15]:
from google.colab import files
uploaded = files.upload()

Saving Dataset.yolov4pytorch.zip to Dataset.yolov4pytorch (1).zip
Saving sam_vit_b_01ec64.pth to sam_vit_b_01ec64 (1).pth


In [25]:
sam_checkpoint = "sam_vit_b_01ec64.pth"

In [26]:
zip_path = "Dataset.yolov4pytorch.zip"
extract_dir = "/content/dataset"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [27]:
model_type = "vit_b"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device)
sam.train()

Sam(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
    )


In [28]:
predictor = SamPredictor(sam)


4.

In [29]:
def grayscale(image: np.ndarray) -> np.ndarray:
    if len(image.shape) == 3:
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    else:
        return image


In [30]:
def equalize(image: np.ndarray) -> np.ndarray:
    image = np.array(image, dtype=np.uint8)
    image = np.ascontiguousarray(image)

    if len(image.shape) == 3:
        yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
        yuv[:, :, 0] = cv2.equalizeHist(yuv[:, :, 0])
        return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)
    else:
        return cv2.equalizeHist(image)

In [31]:
def clahe(image: np.ndarray, clip_limit: float = 2.0, tile_grid_size=(8, 8)) -> np.ndarray:
    image = np.array(image, dtype=np.uint8)
    image = np.ascontiguousarray(image)

    clahe_filter = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid_size)

    if len(image.shape) == 3:
        lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
        lab[:, :, 0] = clahe_filter.apply(lab[:, :, 0])
        return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    else:
        return clahe_filter.apply(image)

In [32]:
def sharpen(image: np.ndarray, strength: float = 1.0, kernel_type: str = 'laplacian') -> np.ndarray:
    image = np.asarray(image, dtype=np.uint8)

    if kernel_type == 'laplacian':
        kernel = np.array([
            [0, -1, 0],
            [-1, 5, -1],
            [0, -1, 0]
        ], dtype=np.float32)
        kernel = kernel * strength
        kernel[1, 1] = 4 * strength + 1
        kernel = np.ascontiguousarray(kernel, dtype=np.float32)
        return cv2.filter2D(image, -1, kernel)

    elif kernel_type == 'unsharp':
        gaussian = cv2.GaussianBlur(image, (0, 0), 2.0)
        return cv2.addWeighted(image, 1.0 + strength, gaussian, -strength, 0)

    else:
        raise ValueError("kernel_type deve ser 'laplacian' ou 'unsharp'")

In [46]:
class ImagePreprocessor:
    def __init__(self, dataset_dir="/content/dataset"):
        self.dataset_dir = dataset_dir

    def process_images(self):

        for subdir in ["train", "valid"]:
            folder_path = os.path.join(self.dataset_dir, subdir)
            if os.path.exists(folder_path):
                print(f"Processing images in: {folder_path}")
                for filename in os.listdir(folder_path):
                    image_path = os.path.join(folder_path, filename)

                    if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
                        print(f"Processing image: {image_path}")
                        processed_img = self.apply_filters(image_path)
                        if processed_img is not None:

                            yield processed_img, image_path
            else:
                print(f"Directory not found: {folder_path}")

    def apply_filters(self, image_path):
        try:
            img = cv2.imread(image_path)

            if img is None:
                print(f"Could not read image: {image_path}")
                return


            gray_img = grayscale(img)
            #equalized_img = equalize(gray_img)
            #clahe_img = clahe(equalized_img)
            #sharpened_img = sharpen(equalized_img, strength=1.0, kernel_type='laplacian')

            print(f"Applied filters to {image_path}")

            return gray_img
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")


preprocessor = ImagePreprocessor()
preprocessor.process_images()

<generator object ImagePreprocessor.process_images at 0x7d34de849ad0>

In [ ]:
def detectar_fissuras(image: np.ndarray, predictor: SamPredictor, threshold_area=100):
    # Ensure the input image is a NumPy array of the correct type
    if not isinstance(image, np.ndarray):
        print("Error: Input 'image' must be a NumPy array.")
        return [] # Return empty list if input is not an image

    # Convert grayscale images to RGB if necessary for SAM predictor
    if len(image.shape) == 2:
        image_rgb = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    else:
        image_rgb = image # Assume it's already RGB or BGR (SAM expects RGB)


    predictor.set_image(image_rgb) # Set the image for the predictor

    # For this example, let's use the center point as an initial prompt
    # Ensure point is within image bounds
    center_x = image_rgb.shape[1] // 2
    center_y = image_rgb.shape[0] // 2
    input_point = np.array([[center_x, center_y]])
    input_label = np.array([1])

    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    resultados = []
    for i, mask in enumerate(masks):
        # Calculate bounding box
        y_indices, x_indices = np.where(mask)
        if len(x_indices) == 0 or len(y_indices) == 0:
            continue
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()

        # Filter small areas (adjust threshold as needed)
        # Calculate area based on bounding box size
        bbox_area = (x_max - x_min) * (y_max - y_min)
        # Alternatively, use the actual mask area:
        mask_area = np.sum(mask)

        # Choose which area to filter by. Bbox area is simpler, mask area is more accurate.
        # Let's use mask_area for more precise filtering.
        if mask_area > threshold_area:
             resultados.append({
                'mask_id': i,
                'bbox': (x_min, y_min, x_max, y_max),
                'score': scores[i],
                'mask': mask
            })

    return resultados

# Create an instance of the preprocessor
preprocessor = ImagePreprocessor()

# Use the generator to get processed images one by one
for processed_image, original_path in preprocessor.process_images():
    if processed_image is not None: # Ensure the image was processed successfully
        print(f"\nDetecting fissures in processed image from: {original_path}")

        # Call detectar_fissuras with the processed image (NumPy array)
        resultados = detectar_fissuras(processed_image, predictor)

        # Display results for the current image
        if resultados:
            print(f"{len(resultados)} fissure(s) detected in {original_path}")

            # Make a copy of the image to draw on
            image_with_bboxes = processed_image.copy()

            # Ensure the image is 3 channels for drawing rectangles if it's grayscale
            if len(image_with_bboxes.shape) == 2:
                image_with_bboxes = cv2.cvtColor(image_with_bboxes, cv2.COLOR_GRAY2RGB)


            # Visualização
            for r in resultados:
                x_min, y_min, x_max, y_max = r['bbox']
                # Draw rectangle on the image copy
                cv2.rectangle(image_with_bboxes, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2) # Green color, thickness 2

            plt.figure(figsize=(10, 10))
            plt.imshow(image_with_bboxes) # Show the image with bounding boxes
            plt.title(f'{len(resultados)} fissura(s) detectada(s) in {os.path.basename(original_path)}')
            plt.axis('off')
            plt.show()

            # Imprimir coordenadas das fissuras detectadas
            for i, r in enumerate(resultados):
                print(f"Fissura {i+1}: Coordenadas da bounding box = {r['bbox']}, Score = {r['score']:.2f}")
        else:
            print(f"No fissures detected in {original_path}")

In [35]:
def detectar_fissuras(image, predictor, threshold_area=100):
    predictor.set_image(image)

    # Para este exemplo vamos usar o ponto central como prompt inicial
    input_point = np.array([[image.shape[1] // 2, image.shape[0] // 2]])
    input_label = np.array([1])

    masks, scores, logits = predictor.predict(
        point_coords=input_point,
        point_labels=input_label,
        multimask_output=True,
    )

    resultados = []
    for i, mask in enumerate(masks):
        # Calcula bounding box
        y_indices, x_indices = np.where(mask)
        if len(x_indices) == 0 or len(y_indices) == 0:
            continue
        x_min, x_max = x_indices.min(), x_indices.max()
        y_min, y_max = y_indices.min(), y_indices.max()

        # Filtrar pequenas áreas (ajustar threshold conforme necessário)
        area = (x_max - x_min) * (y_max - y_min)
        if area > threshold_area:
            resultados.append({
                'mask_id': i,
                'bbox': (x_min, y_min, x_max, y_max),
                'score': scores[i],
                'mask': mask
            })

    return resultados

# Exemplo de uso:
resultados = detectar_fissuras(preprocessor, predictor)

# Visualização
for r in resultados:
    x_min, y_min, x_max, y_max = r['bbox']
    cv2.rectangle(preprocessor, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

plt.figure(figsize=(10, 10))
plt.imshow(preprocessor)
plt.title(f'{len(resultados)} fissura(s) detectada(s)')
plt.axis('off')
plt.show()

# Imprimir coordenadas das fissuras detectadas
for i, r in enumerate(resultados):
    print(f"Fissura {i+1}: Coordenadas da bounding box = {r['bbox']}, Score = {r['score']:.2f}")


AttributeError: 'ImagePreprocessor' object has no attribute 'shape'